In [6]:
%run CompilerTheory_Ch5(Tree_Classes).ipynb

In [7]:
%run Lexical_Analyzer.ipynb

In [8]:
%run parser_helpers.ipynb

In [9]:
# verbose and very verbose for printing out intermediate steps
v = False
vv = False

In [10]:
def big_tree(root):
    """combines the top two subtrees on the TS stack into a bigger tree"""
    global TS
    
    T = MutableLinkedTree()            # --------------------------------Create sub-tree
    root = T.add_root(root)            # ----------------------------------Add root node 
    right = TS.pop()                   # ------------------------------pop right subtree
    left = TS.pop()                    # -------------------------------pop left subtree
    T._attacho(root,left)              # ------------------------add children to subtree
    T._attacho(root,right)
    TS.push(T)                         #  ----------------------push sub-tree onto stack

In [11]:
def parser(m):
    """function to move through the tokens and stack of productions to determine if the input matches the grammar
    and generate an abstract syntax tree"""
    global S, PT, P, t, num_errors, ST, TS, op, t2
    
    dec_flag = False                                # flag for if a variable is being declared
    pre = False                                     # flags for pre and post unary operators
    post = False
    have_moved_on = True                            # flag to avoid repeatedly adding stmts to stack before moving to the next token
    codeblock_flag = False                          # flag for making codeblock subtrees inside curly brackets 

    init_LA()                                       # initialize the lexical analyzer 
    init_parse()                                    # initialize the parse table, stack, production list
    
    t = lexan(m)                                    # run the lexical analyzer to get tokens
    t2 = lexan(m)                                   # get a peak token
    if v: print("initial tokenvalue = ", t.tokenval)
        
    if t.tokentype == 'ID' and not t.dec_flag:      # find declaration errors when the ID is the first token
        print('Error: variable', t.tokenval, 'used on line', line_num, 'before declared.')
        num_errors += 1                             # keep track of numer of errors
    
    while True:  
        
        #---------------------------------------------------------------------set up lookahead and stacks
        if v: print("start of while loop stack:", S.print_s(),"\n")
        if t.tokentype == "OP" or t.tokentype == "RELOP" or t.tokentype == "PUN" or t.tokenval == "$":
            lookahead = t.tokenval                  # tokenval is in PT for these types 
        else:
            lookahead = t.tokentype                  # otherwise, we want the token types for the PT
        if v: print("lookahead = ", lookahead)    
        try:
            non_term = S.pop()                      # non_term is the current non-terminal on top of the stack
        except IndexError:
            print("\nPlease fix errors")            # the stack was emptied while trying to recover from an error
            return                          
        if v: print("non-term = ", non_term)
            
        if lookahead in ['IF', 'ELSE', 'FOR', 'WHILE', 'DO'] and have_moved_on: 
            stmts.push(lookahead)   # save the statement type in a stack to use as a root later
            have_moved_on = False   # flag is false until we find a match so we don't keep adding the same token to stmts before moving to the next token  
            if vv: print('stmts:', stmts.print_s())
        #---------------------------------------------------------------------end of set up section
        
        
        #---------------------------------------------------------------------checking for matches
        #------------------------------------------------------------checking for done symbol
        if non_term == lookahead:                   # check if the lookahead matches the top value on the stack
            if non_term == "$":                     # if that value also matches the end symbol, we are done
                if num_errors == 0:                 # check if there were any errors and print the corresponding message
                    print("\nParsing Complete")
                    break                           # end the while loop and construct the final tree
                else:
                    print("\nPlease fix errors")
                    #T = MutableLinkedTree() 
                    #T.add_root(':(')
                    #return T
                    return
        #----------------------------------------------------------end of done symbol section
            else:                                   # otherwise, we have a match
        #-------------------------------------------------------------------make trees section
                if lookahead in ['(','['] :                        # check for open parenthesis and brackets
                    op.push(lookahead)
                    if vv: print("open parenthesis. op:", op.print_s())
                        
                if lookahead == ')':
                    if op.top() == '(':                            # if we match open and closed paren
                        op.pop()                                   # get rid of the open paren
                        if vv: print("parentehsis matching. op:", op.print_s())
                        # check if there are more arithmetic operators that were inside the parentheses
                        if op and op.top() not in ['(', '=', '+=', '-=', '*=', '/=']:   
                            big_tree(op.pop())
                            
                    # check end of parenthesis for last assign in FOR () - need the next token to be { 
                    if op and op.top() in ['=', '+=', '-=', '*=', '/='] and t2.tokenval == '{':  
                        big_tree(op.pop())                    
                
                    # handle the end of a DO-WHILE loop
                    if t2.tokenval == ';' and stmts.top() == 'WHILE':
                        big_tree('DO-WHILE')
                        stmts.pop()                          # pop the 'DO' stmt off of the stack
                        
                if lookahead == '{':                         # at the beginning of a code block
                    T=MutableLinkedTree() 
                    T.add_root('codeblock')                  # add a codeblock subtree to the stack
                    TS.push(T)
                    
                if lookahead == '}':                         # check for end of code block in If, For, While statements
                    temp = TS.pop()                          # store the top subtree that will not be the 'codeblock' subtree
                    if TS.top()._root._element != 'codeblock':    # check for a second statement subtree on the stack
                        codeblock_flag = True
                        T = MutableLinkedTree()              # add a code parent to the first subtree
                        root = T.add_root('code')                       
                        T._attacho(root,temp) 
                        temp = T                             # store the new subtree
                    else:                                    # there was only one statement in the codeblock
                        TS.pop()                             # pop off the 'codeblock' tree from the stack
                        TS.push(temp)                        # put the statement subtree back on the stack
                    while codeblock_flag:                    # while we have not reached the end of the statements inside the codeblock
                        if TS.top()._root._element != 'codeblock':     # check that there were two subtrees that were not the codeblock tree
                            T = MutableLinkedTree()                # the parent of the statement tree is 'code'
                            root = T.add_root('code')
                            left = TS.pop()                        # pop off the left child
                            T._attacho(root,left)                  # attatch the children               
                            T._attacho(root,temp)                  # temp would have been the right child
                            temp = T                               # store the resulting tree                            
                        else:                                # the top subtree is the 'codeblock' subtree
                            codeblock_flag = False           # end the while loop
                            TS.pop()                         # remove the 'codeblock' tree
                            TS.push(temp)                    # push the completed tree back on the stack
                            
                    if stmts.top() == 'FOR':                 # attatch four subtrees for FOR loops
                        T = MutableLinkedTree()              # create subtree   
                        root = T.add_root(stmts.pop())      
                        child1 = TS.pop()                    # add four children to the root node in reverse order
                        child2 = TS.pop()
                        child3 = TS.pop()
                        child4 = TS.pop()
                        T._attacho(root,child4) 
                        T._attacho(root,child3)
                        if child2._root._element in ['++', '--']:    # if we have ++/--x, push ++/--x before code block
                            T._attacho(root,child2)
                            T._attacho(root,child1)
                        else:                                # otherwise, do the code block before the assing statement
                            T._attacho(root,child1)
                            T._attacho(root,child2)
                        TS.push(T) 
                    elif stmts.top() == 'ELSE':              # add the ELSE tree to the FOR tree as a third child 
                        
                        else_tree = TS.pop()                 # get the ELSE subtree from the stack            
                        if_tree = TS.pop()                   # get the IF subtree from the stack 
                        
                        T = MutableLinkedTree()
                        root = T.add_root('ELSE')
                        if_tree._attacho(if_tree.root(),T)    # attach the ELSE subtree    
                        T._attacho(root,else_tree)    # attach the ELSE subtree     
                        TS.push(if_tree)
                    elif stmts.top() == 'DO':
                        pass                                 # make the DO-WHILE tree after the while code block
                    else: 
                        big_tree(stmts.pop())                # for other statement types, add only two subtrees
                        if vv: print("} TS:", TS.print_ts()) 
                        
                if lookahead == ';':                         # check for end of statement
                    while len(op) and op.top() != '(':       # if there are non-open parenthesis operators, make trees
                        if vv: print("while loop op:", op.print_s())
                        big_tree(op.pop())
                        if vv: print("while loop TS:", TS.print_ts())                       
                            
                if t.tokentype in ['OP', 'RELOP'] and t.tokenval not in ['++','--']:    # put non-unary operators on the operator stack
                    op.push(lookahead)
                    if vv: print('pushing op:', op.print_s())
                        
                if lookahead in ['ID', 'NUM']:               # make one-node trees for ID and NUM tokens
                    if vv: print("ID/NUM. op:", op.print_s())
                    T = MutableLinkedTree()                          
                    T.add_root(t.tokenval)                 
                    TS.push(T)                            
                    if vv: print("ID/NUM. TS:", TS.print_ts())
                    # check for left-associative operators
                    if (t2.tokenval in ['*', '/', '%'] and op and op.top() in ['*', '/', '%']) or (t2.tokenval in ['+', '-'] and op and op.top() in ['+', '-']):
                        if vv: print("big if op:", op.print_s())
                        big_tree(op.pop())
                        if vv: print("big if TS:", TS.print_ts())
                    if t2.tokenval in ['+', '-'] and op and op.top() in ['*', '/', '%']:
                        if vv: print("big if op:", op.print_s())
                        big_tree(op.pop())
                        if vv: print("big if TS:", TS.print_ts())
                    if t2.tokenval in ['++', '--']:       # check for unary operators post ID without array (grammar doesn't have with array)
                        post = True                       # flag for if the unary op is pre or post
                    if pre:                               # we had ++/--ID
                        pre = False                       # reset the flag
                        ID_tree = TS.pop()                # get the ID tree from the top of the stack
                        op_tree = TS.pop()                # next on the stack is the unary operator tree
                        op_tree._attacho(op_tree.root(), ID_tree)   # attach the ID to the bottom of the op tree
                        TS.push(op_tree)
                        
                if lookahead in ['++', '--']:             # check for unary operators
                    if post:                              # we have ID++/--
                        post = False                      # reset the flag 
                        ID_tree = TS.pop()                # get the ID tree from the top of the stack
                        T = MutableLinkedTree()           # make a tree for the op
                        root = T.add_root(lookahead) 
                        ID_tree._attacho(ID_tree.root(), T)    # attatch the op to the bottom of the ID tree
                        TS.push(ID_tree)
                    else:                                 # we have ++/--ID
                        pre = True
                        T = MutableLinkedTree()           # make a tree for the op
                        root = T.add_root(lookahead) 
                        TS.push(T)
        #-------------------------------------------------------------------end of tree section
        #-----------------------------------------------------------handle declarations section
                if t2.tokentype == 'ID':                  # check if the next token is a variable
                    if dec_flag:                          # if the last token was INT, we are declaring this variable
                        t2.dec_flag = True                # have the variable flagged declared 
                        dec_flag = False                  # reset the flag
                    else:                                 # if the last token was not INT and we are not declaring a variable 
                        if not t2.dec_flag:               # and if the variable has not been declared already, we have an error
                            print('Error: variable', t2.tokenval, 'used on line', line_num, 'before declared.')
                            num_errors += 1
        #----------------------------------------------------------end of declarations section
                match(lookahead, non_term, m)             # move to the next token
                have_moved_on = True                      # set the flag for if we have moved to the next token to true
        #-------------------------------------------------------------------------------end of matching section
        
        
        
        #-------------------------------------------------------------------------expanding productions section
        else:                                       # need to match production rule
            try:                                   
                production = PT[non_term][lookahead]    # get production number from parse table
                if v: print("production number", production)
            #-----------------------------------------errors
            except KeyError:
                bad_symbol(lookahead, non_term, m)     # function for handling a key error
                num_errors += 1                     # found an error
                continue                            # don't try to apply production rule
                        
            if production == 0:
                bad_production(lookahead, non_term, m) # function for not finding a production rule
                num_errors += 1                     # found an error
            #-----------------------------------------end errors
                
            else:
                rule = P[production]                # get production rule from production list
                if rule != None:                    # check if the production sends a non-term to empty 
                    for i in range(0,len(rule)):    # otherwise, push the production onto stack in opposite order
                        S.push(rule[len(rule)-1-i])    
                    if production == 3:             # a variable is being declared
                        dec_flag = True             # declaration flag will be true for the next loop iteration when peak token is an ID
       #------------------------------------------------------------------------------end of expanding section
       #---------------------------------------------------------------making trees when something goes to none
                else:
                    if non_term in ['moreterms', 'morefactors'] and op and op.top() not in ['(', '=', '+=', '-=', '*=', '/=']:
                        if t.tokenval == ']' and op.top() == '[':       # if the next tokenval is ], the expression was part of an array
                            expr_tree = TS.pop()    # get the expression from the top of TS
                            ID_tree = TS.pop()      # get the ID that goes with the array 
                            ID_tree._attacho(ID_tree.root(), expr_tree)    # attach the array expression to the ID tree
                            TS.push(ID_tree)
                            op.pop()                # remove the open bracket
                            if vv: print("after ] TS:", TS.print_ts())
                                
                        elif op.top() != '[':       # create trees inside the array brackets
                            if vv: print("none rule op:", op.print_s())
                            big_tree(op.pop())
                            if vv: print("none rule TS:", TS.print_ts())
        #--------------------------------------------------------end of trees when going to none section 
        #--------------------------------------------------------------------------------end of while loop
      
    #--------------------------------------------------------------------------------create the final AST
    left = TS.pop()                            # the top statement tree will be the last child in the AST
    T = MutableLinkedTree()                    # the parent of the statement tree is 'code'
    root = T.add_root('code')
    T._attacho(root,left)                      # attach the statement to the code treee
    TS.push(T)                                 # push the tree back on the stack
    
    while len(TS) > 1:                         # while we have more than the final tree in the stack
        T = MutableLinkedTree()                # the parent of the statement tree is 'code'
        root = T.add_root('code')
        right = TS.pop()                       # pop the children off
        left = TS.pop()
        T._attacho(root,left)                  # attatch the children               
        T._attacho(root,right)
        TS.push(T)
        
    return TS.pop()                            # return the AST
   